In [766]:
import pandas as pd
import json
import math
import hashlib, binascii
import numpy as np
import time
from shutil import copyfile
from os import listdir
import re

In [771]:
def getFiles():
    dlFiles = listdir("./download")
    try:
        dlFiles.remove('.DS_Store')
    except:
        pass
    
    result = []
    for file in dlFiles:
        fileObject={"name":None, "data":None}
        newData = pd.read_csv("./download/"+file, header=None, names=newColumns)
        fileObject["name"] = file.split(".")[0]
        fileObject["data"] = newData
        result.append(fileObject)
    return result


def getFile(file):
    if file == "data":
        return pd.read_csv("./processed/data.csv", header=None, names=oldColumns,index_col=False)
    elif file == "processed":
        return pd.read_csv("./processed/processed.csv",index_col=False)
    elif file == "maps":
        return pd.read_csv("./rules/1to1maps.csv", header=None, names=['item', 'subCategory'])
    elif file == "subCategories":
        return pd.read_csv("./rules/categories.csv", header=None, names=['item', 'subCategory'])
    elif file == "categories":
        return pd.read_csv("./rules/breakdown.csv", header=None, names=['subCategory', 'category'])

def saveDf(df, fileName, path, header):
    miliTime = int(round(time.time() * 1000))
    df.to_csv(f'./backup/{fileName}-{str(miliTime)}.csv', index=False, header=header)
    df.to_csv(f'./{path}/{fileName}.csv', index=False, header=header)    
    

In [780]:
# Hash Data

oldColumns=['date','item','debit','credit','subCategory','hash', 'account']
addedColumns = ['date','item','debit','credit','hash','account']
newColumns=['date','item','debit','credit','card']
processedColumns=['item','category','subCategory','date','year','month','debit','credit','balance', 'account']

def hashit(df):
    hashs = []
    for index, row in df.iterrows():
        h = hashlib.new('ripemd160')
        it = str(row['item'])
        it2= ' '.join(re.findall(r"[\w']+", it))
        h.update(it2.encode())
        h.update(str(row['credit']).encode())
        h.update(str(row['debit']).encode())
        h.update(str(row['date']).encode())
        hashed =h.hexdigest()
        hashs.append(hashed)
    return hashs

def testDf(df):
    assert df.dtypes['debit'] == 'float64'
    assert df.dtypes['credit'] == 'float64'

def fixDf(df):
    global oldColumns
    df['subCategory'].fillna("",inplace=True)
    df=df[oldColumns]
    return df

def findNewItems(old, new, fileName):
    new['hash']= hashit(new)
    hashfound = []
    for index, row in new.iterrows():
        hashfound.append(row['hash'] in old['hash'].values)
    new['hashfound']=hashfound
    new.loc[new['hashfound'] == False, 'account'] = fileName;
    newItems = new[new['hashfound'] == False]
    return newItems

def writeToJson(df):
    gooddata = df
    items = []
    for index, row in gooddata.iterrows():
        temp = {}
        for key in row.keys():
            temp[key] = row[key]
        items.append(temp)
    with open('analysis/js/data.json', 'w') as jsonFile:
        json.dump(items, jsonFile)

def listNewItems(files):
    global oldColumns
    old = getFile('data')
    old['subCategory'].fillna("",inplace=True)
    old.fillna(value=0,inplace=True)
    testDf(old)

    combinedAll = old[oldColumns]
    combinedNew = pd.DataFrame(columns=oldColumns)

    for new in files:
        newData = new['data']
        newName = new['name']
        newData.fillna(value=0,inplace=True)
        testDf(newData)

        newItems = findNewItems(combinedAll, newData, newName)
        newToSave = newItems[addedColumns]

        print(f"{newName} - {len(newToSave)} new items found")

        combinedAll = pd.concat([combinedAll, newToSave])
        combinedNew = pd.concat([combinedNew, newToSave])
        
    combinedNew = fixDf(combinedNew)
    return combinedNew

# Process Hashed Data
def processData(newItems,doAll = False):
    if doAll:
        data = getFile('data')
    else:
        newItems.reset_index(inplace=True)
        data = newItems.copy()

    maps = getFile('maps')
    subCategories = getFile('subCategories')
    categories = getFile('categories')

    categoryMap ={}
    for i, row in categories.iterrows():
        categoryMap[row['subCategory']] = row['category']

    data.fillna("", inplace=True)

    subCatArray = []
    # first mapping the 1to1 mappings
    for i, row in data.iterrows():
        if (row['subCategory'] != ""):
            subCatArray.append(row['subCategory'])
        else:
            try:
                index = pd.Index(maps['item']).get_loc(row['item'].rstrip())
                subCategory = maps.loc[index]['subCategory']
                subCatArray.append(subCategory)
            except:
                subCatArray.append("")

    # then mapping all the general categories
    data['subCategory'] = subCatArray
    subCatArray = pd.Series(subCatArray) 


    for i, categoryRow in subCategories.iloc[::-1].iterrows():
        indo = ((data['item'].str.contains(categoryRow['item'])) & (data['subCategory']==""))
        subCatArray[indo] = categoryRow['subCategory']

    data['balance']=data['credit']-data['debit']

    # finally taking care of special categories with logic
    specialCategories = subCategories[subCategories['item'].str.contains('{{')]
    for i, categoryRow in specialCategories.iterrows():
        itemValuePair = categoryRow['item'].replace('}}', '').split('{{')
        indo = (data['item'].str.contains(itemValuePair[0].rstrip()) & (data['balance']==(float(itemValuePair[1]))))
        subCatArray[indo] = categoryRow['subCategory']

    data['subCategory'] = subCatArray
    data['category'] = data['subCategory'].map(categoryMap)
    data['year']= pd.to_datetime(data['date']).dt.year
    data['month']= pd.to_datetime(data['date']).dt.month    
    return data

def runProcess(files):
    newItems = listNewItems(files)
    if(newItems['item'].count() > 0):
        processedData = processData(newItems)   
        dataWithoutCategory = (processedData[processedData['subCategory'] == ""])
        if(len(dataWithoutCategory) == 0):
            processedAlready = getFile('processed')
            processedAll = pd.concat([processedData, processedAlready])
            processedToSave = processedAll[processedColumns].sort_values(by='date', ascending=False)
            saveDf(processedToSave, 'processed', 'processed', True)

            dataAll = getFile('data')
            combinedData = pd.concat([dataAll, newItems])
            combinedData = combinedData[oldColumns]
            saveDf(combinedData, 'data', 'processed', False)

            writeToJson(processedToSave)

            print("SAVED")
        else:
            print('Found Gaps, NOT SAVED')
            print(dataWithoutCategory[['item','date','balance']])
    #       dataWithoutCategory[['item','date','balance']].to_csv('./processed/not_found.csv')
    else:
        print('no new items')
        
def resetToCurrentData():
    processedData = processData(None, True)  
    processedToSave = processedData[processedColumns].sort_values(by='date', ascending=False)
    saveDf(processedToSave, 'processed', 'processed', True)


In [781]:
  # files should have schema [{'name':string, 'data':pd.DataFrame}, ...]
files = getFiles()
runProcess(files)


line - 0 new items found
visa2 - 0 new items found
no new items


In [777]:
resetToCurrentData()

In [782]:
# reset data and process from backup
from shutil import copyfile
copyfile("./backup/data.csv", "./processed/data.csv")
copyfile("./backup/processed.csv", "./processed/processed.csv")

'./processed/processed.csv'

In [649]:
# HASHING UTILITIES
# hash the data file
# old = pd.read_csv("./processed/data.csv", header=None,names=oldColumns)
# old['custom'].fillna("",inplace=True)
# old.fillna(value=0,inplace=True)
# assert old.dtypes['debit'] == 'float64'
# assert old.dtypes['credit'] == 'float64'
# old['hash']= hashit(old)
# old.to_csv(f'./processed/data.csv', index=False, header=False)

# hash the other files
# for file in dlFiles:
#     old = pd.read_csv("./processed/cards/"+file, header=None,names=oldColumns)
#     old['custom'].fillna("",inplace=True)
#     old.fillna(value=0,inplace=True)
#     assert old.dtypes['debit'] == 'float64'
#     assert old.dtypes['credit'] == 'float64'
#     old['hash']= hashit(old)
#     old.to_csv(f'./processed/{file.split(".")[0]}.{file.split(".")[1]}', index=False, header=False)

# # populate account field in data
# oldColumns=['date','item','debit','credit','custom','hash', 'account']

# data = pd.read_csv("./processed/data.csv", header=None,names=oldColumns)
# for file in dlFiles:
#     new = pd.read_csv("./processed/"+file, header=None,names=oldColumns)
#     for index, row in new.iterrows():
#         if(row['hash'] in data['hash'].values):
#             data.loc[data['hash'] == row['hash'],'account']=file.split(".")[0]
            
# data.to_csv(f'./processed/data.csv', index=False, header=False) 


# # check for duplicate fields

# data2 = pd.read_csv("./processed/data.csv", header=None,names=oldColumns)
# for index, row in data2.iterrows():
#     if(data2[data2['hash']==row['hash']].count()['hash']>2):
#         print(row['hash'], row['item'], row['debit'])

In [ ]:
# # PROCESS BENCH MARKING
# # TESTING TWO FILES
# old = pd.read_csv("./processed/output-old.csv", index_col=False)
# new = pd.read_csv("./processed/processed.csv", index_col=False)
# old.sort_values(by=['date','debit'], inplace=True)
# new.sort_values(by=['date','debit'], inplace=True)
# old.reset_index( drop=True,inplace=True)
# new.reset_index(drop=True,inplace=True)
# ineq = []
# for i, row in old.iterrows():
#     if (row['subCategory'] != new.loc[i]['subCategory']):
#         if(row['item']=="TIGERDIRECT.CA MARKHAM, ON"):
#             pass
#         elif('WINNERSHOMESENSE' in row['item']):
#             pass
#         else:
#             pass
# #             print(f"{row['subCategory']}, {new.loc[i]['subCategory']} - {row['balance']} {row['item']}{new.loc[i]['item']}")
            

# y = new.groupby('subCategory').sum()['balance']
# x = old.groupby('subCategory').sum()['balance']
# for i,value in enumerate(x):
#     if(value != y[i]):
#         print(value, y[i])

# # one to one map benchmarking
# def try1():    
#     def check1to1(x):
#         try:
#             index = pd.Index(maps['item']).get_loc(x.rstrip())   
#             return maps.loc[index]['subCategory']
#         except:
#             return None

#     data['subCategory'] = data['subCategory'].combine_first(data['item'].apply(check1to1))
    
# def try2():
#     data.fillna("", inplace=True)
#     subCatArray = []
#     # first mapping the 1to1 mappings
#     for i, row in data.iterrows():
#         if (row['subCategory'] != ""):
#             subCatArray.append(row['subCategory'])
#         else:
#             try:
#                 index = pd.Index(maps['item']).get_loc(row['item'].rstrip())
#                 subCategory = maps.loc[index]['subCategory']
#                 subCatArray.append(subCategory)
#             except:
#                 subCatArray.append("")
#     data['subCategory'] = subCatArray
# %timeit try1()
# %timeit try2()